In [1]:
!conda install -yc conda-forge ipython-sql

/bin/sh: 1: conda: not found


In [3]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

In [9]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


# Overview of the Data

In [10]:
%%sql
select * from facts
limit 5

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


# Summary Statistics

In [12]:
%%sql
select min(population), max(population), min (population_growth),max(population_growth)
from facts

Done.


min(population),max(population),min (population_growth),max(population_growth)
0,7256490011,0.0,4.02


# Exploring Outliers

A few things are interesting in the summary statistics on the previous screen:

* There's a country with a population of 0.
* There's a country with a population of 7256490011 (or more than 7.2 billion people).
Let's use subqueries to concentrate on these countries without using the specific values.

In [13]:
# Write a query that returns the countries with the minimum population.
%%sql
select name from facts
where population = (select min(population) from facts)

Done.


name
Antarctica


In [19]:

%%sql
select * from facts
where population = (select max(population) from facts)

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


We also see that the table contains a row for the whole world, which explains the maximum population of over 7.2 billion we found earlier.

Now that we know this, we should recalculate the summary statistics we calculated earlier, while excluding the row for the whole world.

# Summary Statistics Revisited

In [22]:
%%sql
select min(population), max(population), 
    min (population_growth),max(population_growth)
from facts
where name != 'World'

Done.


min(population),max(population),min (population_growth),max(population_growth)
0,1367485388,0.0,4.02


# Exploring Average Population and Area

In [23]:
%%sql
select avg(population), avg(population) 
from facts
where name != 'World'

Done.


avg(population),avg(population)_1
32242666.56846473,32242666.56846473


# Finding Densely Populated Countries

to find countries that are densely populated. We'll identify countries that have the following:

* Above-average values for population.
* Below-average values for area.

In [ ]:
%%sql
select * from facts
where population > (select avg(population) from facts)
    and are